In [308]:
import pandas as pd
import numpy as np
import calendar

import os
import re
import string

from datetime import datetime, date

In [309]:
from download_data import all_data
file_format = os.path.join("data", "{}_{}_{}.xls")
files = [file_format.format(m, y, t) for m, y, t in all_data if os.path.exists(file_format.format(m, y, t))]

In [310]:
pollen_sheets = [pd.read_excel(file, skiprows=4, skip_footer=1).dropna(axis=(0,1), how="all") for file in files if file.endswith("pollen.xls")]
pollen_sheets[0].head()

,DATE,Ash,Ashe Juniper / Bald Cypress,Elm,Pine,Tree Total,Grass Total,Tree & Grass Total,Weed Total,POLLEN TOTAL,Tech.
0,1,NaN,NaN,NaN,NaN,0,NaN,0,0,0,NC
1,2,NaN,6.0,NaN,NaN,6,NaN,6,0,6,GG
2,3,NaN,2.0,NaN,NaN,2,NaN,2,0,2,GG
3,4,NaN,NaN,2.0,NaN,2,NaN,2,0,2,GG
4,5,NaN,NaN,NaN,NaN,0,NaN,0,0,0,Weekend


In [311]:
# Load mold sheets, and transpose because the header is by row
mold_sheets = [pd.read_excel(file, skiprows=5, skip_footer=7).transpose() for file in files if file.endswith("mold.xls")]

# we have to make the first row the columns because we transposed
def fix_mold_header_and_drop(sheet):
    header = sheet.iloc[0]
    sheet.columns = header
    sheet = sheet.drop(sheet.index[0])
    sheet = sheet.reset_index()
    sheet = sheet.rename(columns={"index": "Date"})
    return sheet
mold_sheets = list(map(fix_mold_header_and_drop, mold_sheets))
mold_sheets[0]['Date']

0        1
1        2
2        3
3        4
4        5
5        6
6        7
7        8
8        9
9       10
10      11
11      12
12      13
13      14
14      15
15      16
16      17
17      18
18      19
19      20
20    20.1
21      22
22      21
23    22.1
24      23
25      24
26      25
27      26
28      27
29      28
30      29
31      30
32      31
Name: Date, dtype: object

In [312]:
# Cleaning up and unifying column names
unify_pollen = {
    "Other Tree Pollen": "Other Tree",
    "Other Tree/unidentified": "Other Tree",
    "Other Weed Pollen": "Other Weed",
    "Other Weed/unidentified": "Other Weed"
}
def pollen_column_mapper(column_name):
    # This deletes parens and contents
    result = re.sub(r"\(.+\)", "", column_name)
    result = string.capwords(result)
    result = result.strip()
    result = unify_pollen.get(result, result)
    return result

pollen_sheets = [sheet.rename(columns=pollen_column_mapper) for sheet in pollen_sheets]

In [313]:
unify_mold = {
    "Misc. Fungus (Hyaline)": "Hyaline",
    "*D. Conidia/Hyphae": "Dematiaceous",
    "DATE": "Date"
}
def mold_column_mapper(column_name):
    result = re.sub(r"\(.+\)", "", column_name)
    result = result.title()
    result = result.strip()
    result = unify_mold.get(result, result)
    return result

mold_sheets = [sheet.rename(columns=mold_column_mapper) for sheet in mold_sheets]
mold_sheets[0].head()

DATE,Date,Acrodictys,Agrocybe,Algae,Alternaria,Arthimium,Ascomycetes,Asperisporium,Basidiomycetes,Beltrania,...,Pithomyces,Powdery Mildew,Pseudocercospora,Puccinia,Rust,Spegazinia,Stemphyllium,Tetrapola,Tilletia,Torula
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,10,24,NaN,488,NaN,223,NaN,...,6,NaN,NaN,NaN,NaN,NaN,18,NaN,NaN,NaN
2,3,NaN,NaN,NaN,6,NaN,136,NaN,43,NaN,...,4,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN
3,4,NaN,NaN,16,20,NaN,142,NaN,89,NaN,...,2,NaN,NaN,NaN,NaN,2,6,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [314]:
def drop_total_tech_entries(sheet):
    result = sheet.drop(columns=list(sheet.filter(regex="(Total|Tech)")))
    if any(result['Date'].astype(str) == "Total"):
        result = result[sheet['Date'].astype(str) != "Total"]
    if any(result['Date'].astype(str) == "TOTAL"):
        result = result[result['Date'].astype(str) != "TOTAL"]
    return result
pollen_sheets = list(map(drop_total_tech_entries, pollen_sheets))
mold_sheets = list(map(drop_total_tech_entries, mold_sheets))

In [315]:
month_name_to_number = {v.lower(): k for k,v in enumerate(calendar.month_name)}
def convert_dates(metadata, sheet):
    month = month_name_to_number[metadata[0]]
    year = metadata[1]
    num_days = calendar.monthrange(year, month)[1]
    sheet = sheet[sheet['Date'].astype(str).apply(lambda d: d.isdecimal())]
    sheet = sheet[(sheet['Date'].astype(int) <= num_days) & (sheet['Date'].astype(int) > 0)]
    sheet['Date'] = sheet['Date'].apply(lambda day: date(year, month, int(float(day))))
    return sheet.set_index('Date')
pollen_sheets = list(map(lambda a: convert_dates(a[0], a[1]), zip([a for a in all_data if a[2] == "pollen"], pollen_sheets)))
mold_sheets = list(map(lambda a: convert_dates(a[0], a[1]), zip([a for a in all_data if a[2] == "mold"], mold_sheets)))

In [316]:
from functools import reduce
pollen_sheet = reduce(lambda a,b: a.append(b), pollen_sheets).dropna(axis=(0,1), how="all")
mold_sheet = reduce(lambda a,b: a.append(b), mold_sheets).dropna(axis=(0,1), how="all")

In [317]:
drop_cols = pollen_sheet[pollen_sheet > 0].count(numeric_only=True) <= 1
pollen_sheet = pollen_sheet.drop(columns=drop_cols[drop_cols == True].keys())

In [318]:
drop_cols = mold_sheet[mold_sheet > 0].count(numeric_only=True) <= 1
mold_sheet = mold_sheet.drop(columns=drop_cols[drop_cols == True].keys())

In [319]:
pollen_sheet.describe(percentiles=[.99, .95, .94])

,Amaranth,Ash,Ashe Juniper / Bald Cypress,Birch,Black Walnut,Cedar,Cotton Wood,Elm,Hackberry,Hickory,...,Other Weed,Pine,Plantago,Ragweed,Rumex,Sagebrush,Sedge,Sweet Gum,Sycamore,Willow
count,178.000000,134.000000,212.000000,104.000000,10.000000,18.000000,81.000000,222.000000,54.000000,56.000000,...,62.000000,120.000000,27.000000,306.000000,39.000000,63.000000,160.000000,46.000000,37.000000,43.000000
mean,7.460674,112.537313,74.155660,35.750000,24.200000,76.777778,11.839506,39.990991,33.814815,13.392857,...,7.483871,192.408333,4.518519,107.915033,5.743590,8.650794,8.306250,18.304348,24.000000,30.046512
std,7.478868,194.982346,145.523859,43.242093,32.625144,96.557545,11.429629,81.905971,29.384294,22.804847,...,6.134402,410.917144,5.989545,163.051373,6.176006,7.827080,6.995953,26.864987,34.088121,45.662297
min,0.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
50%,4.000000,49.000000,24.000000,18.000000,7.000000,29.000000,8.000000,10.000000,24.000000,6.000000,...,4.000000,47.000000,2.000000,28.000000,4.000000,6.000000,6.000000,10.000000,10.000000,12.000000
94%,18.000000,428.260000,226.760000,102.560000,81.960000,247.800000,30.400000,156.000000,87.480000,35.800000,...,17.360000,907.300000,11.280000,421.000000,17.760000,25.120000,24.000000,45.900000,92.160000,124.800000
95%,20.000000,446.250000,261.450000,104.850000,84.300000,256.700000,32.000000,169.600000,92.100000,43.500000,...,19.800000,917.800000,14.400000,454.000000,20.400000,27.600000,24.000000,51.750000,96.800000,150.000000
99%,36.760000,877.790000,594.500000,187.220000,93.660000,296.140000,43.200000,387.180000,126.820000,119.700000,...,28.680000,1505.880000,26.880000,724.950000,27.720000,32.280000,30.000000,124.100000,139.680000,174.760000
max,64.000000,1506.000000,1218.000000,279.000000,96.000000,306.000000,72.000000,749.000000,130.000000,123.000000,...,36.000000,3402.000000,30.000000,812.000000,30.000000,36.000000,32.000000,170.000000,162.000000,184.000000


In [328]:
# I think this drops non numeric things ???
mold_sheet = mold_sheet[mold_sheet.astype(str).apply(lambda s: s.apply(lambda d: d.isdecimal()))].astype('float64')

In [329]:
mold_sheet.describe(percentiles=[.99, .95, .94])

,Algae,Alternaria,Ascomycetes,Basidiomycetes,Cercospora,Cladosporium,Curvularia,Drechslera/Helmintho.,Epicoccum,Fusariella,...,Pithomyces,Pleospora,Powdery Mildew,Rust,Spegazinia,Speggazinia,Stemphyllium,Tetrapola,Tilletia,Torula
count,2.000000,4.000000,4.000000,4.000000,4.000000,4.00000,4.000000,4.000000,3.000000,0.0,...,4.000000,0.0,3.00,3.000000,1.0,0.0,1.0,0.0,0.0,2.000000
mean,37.000000,78.250000,2152.000000,682.750000,89.000000,3296.25000,72.500000,44.500000,16.000000,NaN,...,10.000000,NaN,4.00,6.666667,6.0,NaN,2.0,NaN,NaN,21.000000
std,29.698485,48.071995,1244.794762,298.007131,84.750615,1586.60631,51.958317,15.864005,10.392305,NaN,...,6.324555,NaN,2.00,4.163332,NaN,NaN,NaN,NaN,NaN,1.414214
min,16.000000,42.000000,1071.000000,421.000000,20.000000,1597.00000,24.000000,24.000000,4.000000,NaN,...,4.000000,NaN,2.00,2.000000,6.0,NaN,2.0,NaN,NaN,20.000000
50%,37.000000,61.000000,2067.500000,626.500000,62.000000,3184.50000,60.000000,48.000000,22.000000,NaN,...,9.000000,NaN,4.00,8.000000,6.0,NaN,2.0,NaN,NaN,21.000000
94%,55.480000,133.700000,3337.380000,1008.400000,186.800000,4976.36000,131.240000,57.640000,22.000000,NaN,...,16.920000,NaN,5.76,9.760000,6.0,NaN,2.0,NaN,NaN,21.880000
95%,55.900000,136.250000,3348.150000,1016.500000,191.000000,5016.80000,133.700000,57.700000,22.000000,NaN,...,17.100000,NaN,5.80,9.800000,6.0,NaN,2.0,NaN,NaN,21.900000
99%,57.580000,146.450000,3391.230000,1048.900000,207.800000,5178.56000,143.540000,57.940000,22.000000,NaN,...,17.820000,NaN,5.96,9.960000,6.0,NaN,2.0,NaN,NaN,21.980000
max,58.000000,149.000000,3402.000000,1057.000000,212.000000,5219.00000,146.000000,58.000000,22.000000,NaN,...,18.000000,NaN,6.00,10.000000,6.0,NaN,2.0,NaN,NaN,22.000000
